In [ ]:
import numpy as np
import symmetry
import crystal_loader
import pandas as pd

fpath = "../data/TiO2/"

structs, labels, lattice_vectors, counts = crystal_loader.get_dataset(fpath, halt=-1)

In [ ]:
E_O = -432.503149303
E_Ti = -1604.604515075

E_leg = {"Ti": E_Ti, "O": E_O}

In [ ]:
cohesive_e = crystal_loader.get_cohesive_energies(labels, counts, E_leg)
n_atoms = counts.sum(axis=1)

In [ ]:
Rc = 6.5 # A
Rs = 0

In [ ]:
# build radial basis params:

radial_etas = [0.003214, 0.035711, 0.071421, 0.124987, 0.214264, 0.357106, 0.714213, 1.428426]
params_radial = tuple([{"Rs": Rs, "eta": eta} for eta in radial_etas])

In [ ]:
angular_etas = [0.000357, 0.028569, 0.089277, 0.000357, 0.028569, 0.089277, 0.000357, 0.028569, 0.089277] * 2
angular_lambdas = [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
angular_zetas = [1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 4.0]

params_angular = tuple([{"Rs": Rs, "eta": eta, "lmbda": lam, "zeta": zeta} for eta, lam, zeta in zip(angular_etas, angular_lambdas, angular_zetas)])

In [ ]:
tio_params_ang = {("Ti", "Ti"): params_angular, ("Ti", "O"): params_angular, ("O", "O"): params_angular}
tio_params_rad = {"Ti": params_radial, "O": params_radial}

features = crystal_loader.build_features(structs, lattice_vectors, Rc=6.5, params_rad=tio_params_rad, params_ang=tio_params_ang)

In [ ]:
label_df = pd.DataFrame(np.hstack([labels, cohesive_e]), columns=["total_energy", "cohesive_energy"])


In [ ]:
dset_name = "TiO2_2015_angfixed_x3"

import h5py

save_name = f"../pickles/{dset_name}_features.h5"

with h5py.File(save_name, "w") as f:
    for i, arr in enumerate(features):
        f.create_dataset(f"array_{i}", data=arr)

save_name = f"../pickles/{dset_name}_labeldata.h5"

label_df.to_hdf(save_name, key="labels")
counts.to_hdf(save_name, key="atom_counts")
n_atoms.to_hdf(save_name, key="n_atoms")